In [47]:
import h3
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from torch.nn import DataParallel

In [2]:
untrained_model_path = "/home/uddeshya.singh/Un-trained_Model/distilroberta-base/"
retrained_lm_model_save_dir = '/home/uddeshya.singh/Trained_model/save/'
retrained_lm_model_out_dir = '/home/uddeshya.singh/Trained_model/out/'
model_dir = "/home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir"

In [3]:
address_data_path = "/home/uddeshya.singh/DATA/PUNJAB_train/PUNJAB_with_grid_ids_train.csv"

In [4]:
df = pd.read_csv(address_data_path)
df.head()

addr_hash  \
0  bf5547f453b442c509152dc57f4adda3   
1  6a2a44e0fd1e09541cd0be66fc0a0b42   
2  976aff4524ca1b8111447b8d7c93d250   
3  c1ec3b016d41a0ed35c4906f86477695   
4  a2c4ef86c2de9103ea883a27c682ab99   

                                     cleaned_address  lookup_lat  lookup_lng  \
0                               0 abiana nurpur bedi   30.982717   76.522448   
1   0 badi haveli satluj coloney near water suppl...   30.974820   76.513807   
2      0 badi haveli satluj colony near water supply   30.974851   76.513821   
3   0 badi haveli satluj colony near water supply...   30.974481   76.514228   
4                                     0 badi havelli   30.972455   76.512910   

           grid_id  
0  8a3d14732617fff  
1  8a3d14449bb7fff  
2  8a3d14449bb7fff  
3  8a3d14449b97fff  
4  8a3d14449177fff

In [5]:
print('starting now ', time.strftime('%X %x %Z'))
address_data_train = load_dataset("csv", data_files=address_data_path, split='train[:100%]')
print('completed now ', time.strftime('%X %x %Z'))

starting now  19:17:42 07/29/24 IST


Using custom data configuration default-68fb3a491bebfb26
Reusing dataset csv (/home/uddeshya.singh/.cache/huggingface/datasets/csv/default-68fb3a491bebfb26/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


completed now  19:19:02 07/29/24 IST


In [6]:
address_data_train

Dataset({
    features: ['addr_hash', 'cleaned_address', 'lookup_lat', 'lookup_lng', 'grid_id'],
    num_rows: 6913138
})

In [7]:
print('starting now ', time.strftime('%X %x %Z'))
address_data_train = address_data_train.map(lambda x: {'grid_id_str': str(x['grid_id'])}, batched=False)
print('completed now ', time.strftime('%X %x %Z'))

starting now  19:19:02 07/29/24 IST


  0%|          | 0/6913138 [00:00<?, ?ex/s]

completed now  19:28:04 07/29/24 IST


In [8]:
print('starting now ', time.strftime('%X %x %Z'))
address_data_train = address_data_train.class_encode_column("grid_id")
print('completed now ', time.strftime('%X %x %Z'))

starting now  19:28:04 07/29/24 IST


Casting to class labels:   0%|          | 0/6914 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/692 [00:00<?, ?ba/s]

completed now  19:29:19 07/29/24 IST


In [9]:
def get_neighbors(h3_index):
    neighbors_1 = list(h3.k_ring(h3_index, 1))
    neighbors_2 = list(h3.k_ring(h3_index, 2) - set(neighbors_1))
    neighbors_3 = list(h3.k_ring(h3_index, 3) - set(h3.k_ring(h3_index, 2)))
    return neighbors_1, neighbors_2, neighbors_3

In [11]:
neighbors_lookup = {}
label_to_grid_id_str = {}

for idx, row in tqdm(address_data_train.to_pandas().iterrows(), total=address_data_train.num_rows):
    grid_id_str = row['grid_id_str']
    label = row['grid_id']
    neighbors_lookup[grid_id_str] = get_neighbors(grid_id_str)
    label_to_grid_id_str[label] = grid_id_str


100%|██████████| 6913138/6913138 [19:07<00:00, 6024.49it/s]


In [13]:
print("Sample neighbors lookup:")
for key in list(neighbors_lookup.keys())[:1]:
    print(f"{key}: {neighbors_lookup[key]}")


Sample neighbors lookup:
8a3d14732617fff: (['8a3d14732607fff', '8a3d14732617fff', '8a3d1473261ffff', '8a3d14732637fff', '8a3d147326a7fff', '8a3d1473278ffff', '8a3d147326affff'], ['8a3d1473279ffff', '8a3d147326e7fff', '8a3d147326b7fff', '8a3d14732627fff', '8a3d1473260ffff', '8a3d147327affff', '8a3d1473268ffff', '8a3d147326f7fff', '8a3d1473262ffff', '8a3d14732687fff', '8a3d1473271ffff', '8a3d14732787fff'], ['8a3d14732717fff', '8a3d14732757fff', '8a3d14449b4ffff', '8a3d147326c7fff', '8a3d147326d7fff', '8a3d14714d27fff', '8a3d147326effff', '8a3d1473269ffff', '8a3d147327b7fff', '8a3d14732697fff', '8a3d147327a7fff', '8a3d1473275ffff', '8a3d14732797fff', '8a3d14732707fff', '8a3d14449b6ffff', '8a3d14732677fff', '8a3d14732657fff', '8a3d1473270ffff'])


In [14]:
def clean_dataset(batch):
    if pd.isnull(batch['cleaned_address']) or batch['cleaned_address'] == '':
        return False
    return True

In [15]:
print('starting now ', time.strftime('%X %x %Z'))
address_data_train = address_data_train.filter(clean_dataset)
print('completed now ', time.strftime('%X %x %Z'))

starting now  22:42:59 07/29/24 IST


  0%|          | 0/6914 [00:00<?, ?ba/s]

completed now  22:43:59 07/29/24 IST


In [16]:
pretrained_tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [17]:
address_data_train

Dataset({
    features: ['addr_hash', 'cleaned_address', 'lookup_lat', 'lookup_lng', 'grid_id', 'grid_id_str'],
    num_rows: 6913118
})

In [18]:
def tokenize(batch):
    tokenized_batch = pretrained_tokenizer(batch["cleaned_address"], padding=True, truncation=True, max_length=100)
    return tokenized_batch

print('starting now ', time.strftime('%X %x %Z'))
address_data_train = address_data_train.rename_column('grid_id', 'labels')
print('completed now ', time.strftime('%X %x %Z'))

starting now  22:43:59 07/29/24 IST
completed now  22:43:59 07/29/24 IST


In [19]:
print('starting now ', time.strftime('%X %x %Z'))
train_dataset = address_data_train.map(tokenize, batched=True, batch_size=100000, num_proc=7)

starting now  22:43:59 07/29/24 IST
        

#0:   0%|          | 0/10 [00:00<?, ?ba/s]

#1:   0%|          | 0/10 [00:00<?, ?ba/s]

#2:   0%|          | 0/10 [00:00<?, ?ba/s]

#3:   0%|          | 0/10 [00:00<?, ?ba/s]

#4:   0%|          | 0/10 [00:00<?, ?ba/s]

#5:   0%|          | 0/10 [00:00<?, ?ba/s]

#6:   0%|          | 0/10 [00:00<?, ?ba/s]

In [21]:
def custom_collate_fn(batch):
    neighbors_1 = [neighbors_lookup[label_to_grid_id_str[item['labels']]][0] for item in batch]
    neighbors_2 = [neighbors_lookup[label_to_grid_id_str[item['labels']]][1] for item in batch]
    neighbors_3 = [neighbors_lookup[label_to_grid_id_str[item['labels']]][2] for item in batch]

    input_ids = pad_sequence([torch.tensor(item['input_ids']) for item in batch], batch_first=True, padding_value=pretrained_tokenizer.pad_token_id)
    attention_mask = pad_sequence([torch.tensor(item['attention_mask']) for item in batch], batch_first=True, padding_value=0)
    labels = torch.tensor([item['labels'] for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels,
        'neighbors_1': neighbors_1,
        'neighbors_2': neighbors_2,
        'neighbors_3': neighbors_3
    }



In [66]:
train_dataloader = DataLoader(train_dataset, batch_size=12, collate_fn=custom_collate_fn)


In [67]:
# for idx, ele in enumerate(train_dataloader):
#     print(f"Batch {idx}:", ele)
#     break

In [68]:
model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(train_dataset.features['labels'].names))
model.to(device)

Some weights of the model checkpoint at /home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir and are newly initialized: ['classifi

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [69]:
model = DataParallel(model)


In [70]:
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_dataloader) * 10
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [71]:
class NeighborAwareLabelSmoothingLoss(torch.nn.Module):
    def __init__(self, smoothing=0.1, first_order_weight=0.5, second_order_weight=0.3, third_order_weight=0.2):
        super(NeighborAwareLabelSmoothingLoss, self).__init__()
        self.smoothing = smoothing
        self.first_order_weight = first_order_weight
        self.second_order_weight = second_order_weight
        self.third_order_weight = third_order_weight

    def forward(self, output, target, neighbors_1, neighbors_2, neighbors_3):
        num_classes = output.size(-1)
        confidence = 1.0 - self.smoothing
        smoothed_label_dist = torch.full((output.size(0), num_classes), self.smoothing / (num_classes - 1)).to(output.device)
        
        for idx, label in enumerate(target):
            smoothed_label_dist[idx][label] = confidence
            if neighbors_1[idx]:
                weight_1 = self.first_order_weight / len(neighbors_1[idx])
                for neighbor in neighbors_1[idx]:
                    neighbor_idx = int(neighbor, 16) % num_classes  # Ensure neighbor index is valid
                    smoothed_label_dist[idx][neighbor_idx] = weight_1
            if neighbors_2[idx]:
                weight_2 = self.second_order_weight / len(neighbors_2[idx])
                for neighbor in neighbors_2[idx]:
                    neighbor_idx = int(neighbor, 16) % num_classes  # Ensure neighbor index is valid
                    smoothed_label_dist[idx][neighbor_idx] = weight_2
            if neighbors_3[idx]:
                weight_3 = self.third_order_weight / len(neighbors_3[idx])
                for neighbor in neighbors_3[idx]:
                    neighbor_idx = int(neighbor, 16) % num_classes  # Ensure neighbor index is valid
                    smoothed_label_dist[idx][neighbor_idx] = weight_3
        
        log_probs = torch.nn.functional.log_softmax(output, dim=-1)
        loss = (-smoothed_label_dist * log_probs).sum(dim=-1).mean()
        return loss

In [72]:
loss_fn = NeighborAwareLabelSmoothingLoss()


In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DataParallel(
  (module): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear(i

In [74]:
print('starting now ', time.strftime('%X %x %Z'))
model.train()

iteration = 0  # Initialize iteration counter

for epoch in range(10):
    total_loss = 0  # Initialize total loss for the epoch
    
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        logits = outputs.logits
        loss = loss_fn(logits, batch['labels'], batch['neighbors_1'], batch['neighbors_2'], batch['neighbors_3'])

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()  # Accumulate the loss

        # Print loss every 1000 iterations
        if iteration % 1000 == 0:
            print(f"Iteration {iteration} - Loss: {loss.item()}")

        iteration += 1
    
    avg_loss = total_loss / len(train_dataloader)  # Calculate the average loss for the epoch
    print(f"Epoch {epoch+1} completed - Average Loss: {avg_loss}")

print('completed now ', time.strftime('%X %x %Z'))

starting now  23:08:09 07/29/24 IST


  0%|          | 1/576094 [00:00<69:50:51,  2.29it/s]

Iteration 0 - Loss: 24.024887084960938


  0%|          | 96/576094 [00:40<67:43:43,  2.36it/s]


KeyboardInterrupt: 